In [36]:
year = '58'
pdf_filename = 'data/pdf-text' + year + '.txt'
major_filename = 'data/majors-adm' + year + '.csv'
output_filename = 'data/major-scores-adm' + year + '.csv'

In [37]:
lines = open(pdf_filename).readlines()
print(lines[:2])

['ตารางแสดงสถตจานวนผสมคร จานวนรบ จานวนผผานฯ และคะแนนสงสด-ตาสด ของการคดเลอกบคคลเขาศกษาในสถาบนอดมศกษาระบบกลาง (Admissions) ประจาปการศกษา 2558\n', 'รหส\n']


In [38]:
import re
major_number_pattern = r'^\d\d\d\d$'
major_number = re.compile(major_number_pattern)

In [39]:
old_key='xxxx'
old_items = []
pre_results = []
for l in lines:
    for item in l.split():        
        if major_number.match(item):
            pre_results.append((old_key, old_items))
            old_items = []
            old_key = item
        else:
            old_items.append(item)
pre_results.append((old_key, old_items))
print(pre_results[:4])

[('xxxx', ['ตารางแสดงสถตจานวนผสมคร', 'จานวนรบ', 'จานวนผผานฯ', 'และคะแนนสงสด-ตาสด', 'ของการคดเลอกบคคลเขาศกษาในสถาบนอดมศกษาระบบกลาง', '(Admissions)', 'ประจาปการศกษา']), ('2558', ['รหส', 'คณะ/สาขา', 'มหาวทยาลย', 'จานวน', 'คะแนน', 'สมคร', 'รบ', 'ผาน', 'คะแนนสงสด', 'คะแนนตาสด', 'คะแนนเฉลย', 'SD']), ('0001', ['คณะทนตแพทยศาสตร', 'จฬาลงกรณมหาวทยาลย', '265', '16', '16', '23406.2500', '22568.7500', '22833.4563', '243.5615']), ('0002', ['คณะเภสชศาสตร', 'สาขาวชาเภสชกรรมอตสาหการ', 'จฬาลงกรณมหาวทยาลย', '408', '105', '105', '24921.2500', '20738.7500', '21266.0348', '529.1059'])]


In [40]:
import csv
def read_major_title(filename):
    majors = []
    reader = csv.reader(open(filename))
    for row in reader:
        majors.append(row)
    return majors

majors = read_major_title(major_filename)
print(majors[:3])

[['0001', 'คณะทันตแพทยศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ['0002', 'คณะเภสัชศาสตร์ สาขาวิชาเภสัชกรรมอุตสาหการ', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ['0003', 'คณะเภสัชศาสตร์ สาขาวิชาการบริบาลทางเภสัชกรรม', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย']]


In [41]:
result_row = 0
results = []
for mdata in majors:
    number = mdata[0]
    org_result_row = result_row
    while (result_row < len(pre_results)) and (pre_results[result_row][0] != number):
        result_row += 1
    if result_row == len(pre_results):
        result_row = org_result_row
        continue
    results.append((mdata, pre_results[result_row]))
print(results[:3])

[(['0001', 'คณะทันตแพทยศาสตร์', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0001', ['คณะทนตแพทยศาสตร', 'จฬาลงกรณมหาวทยาลย', '265', '16', '16', '23406.2500', '22568.7500', '22833.4563', '243.5615'])), (['0002', 'คณะเภสัชศาสตร์ สาขาวิชาเภสัชกรรมอุตสาหการ', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0002', ['คณะเภสชศาสตร', 'สาขาวชาเภสชกรรมอตสาหการ', 'จฬาลงกรณมหาวทยาลย', '408', '105', '105', '24921.2500', '20738.7500', '21266.0348', '529.1059'])), (['0003', 'คณะเภสัชศาสตร์ สาขาวิชาการบริบาลทางเภสัชกรรม', '0001', 'จุฬาลงกรณ์มหาวิทยาลัย'], ('0003', ['คณะเภสชศาสตร', 'สาขาวชาการบรบาลทางเภสชกรรม', 'จฬาลงกรณมหาวทยาลย', '474', '95', '95', '23001.2500', '20927.5000', '21402.8326', '383.6991']))]


In [42]:
def extract_scores(data):
    if re.match(r'[0-9,]+\.\d+',data[-1]):
        if re.match(r'\d+',data[-7]):
            return data[-7:]
        else:
            return data[-5:]
    else:
        i = -1
        try:
            while not re.match(r'[0-9,]+\.\d+',data[i]):
                #print(i,data[i])
                i -= 1
            #print(">>>",data[i-6])
            if re.match(r'\d+',data[i-6]):
                return data[i-6:i+1]
            else:
                return data[i-4:i+1]
        except:
            if data[-1] == '0':
                return data[-3:]
            else:
                print("error",data)
                return data
        
            
def extract_line(res):
    title = res[0]
    data = res[1][1]
    items = [title[0], title[1], title[3]] + extract_scores(data)
    return items

def print_line(res):
    print(",".join(extract_line(res)))

In [43]:
for r in results[105:115]:
    print_line(r)

0106,คณะมนุษยศาสตร์ สาขาวิชาสื่อสารมวลชน รูปแบบที่ 2 เลือกสอบวิชาภาษาจีน,มหาวิทยาลัยเกษตรศาสตร์,77,0,2,20446.2500,20426.2500,20436.2500,14.1421
0107,คณะมนุษยศาสตร์ สาขาวิชาสื่อสารมวลชน รูปแบบที่ 2 เลือกสอบวิชาภาษาอาหรับ,มหาวิทยาลัยเกษตรศาสตร์,6,0,0
0108,คณะมนุษยศาสตร์ สาขาวิชาสื่อสารมวลชน รูปแบบที่ 2 เลือกสอบวิชาภาษาบาลี,มหาวิทยาลัยเกษตรศาสตร์,89,0,4,20441.2500,19852.5000,20070.4500,257.4992
0109,คณะมนุษยศาสตร์ สาขาวิชาภาษาอังกฤษ ภาคพิเศษ,มหาวิทยาลัยเกษตรศาสตร์,491,100,100,23406.2500,21708.7500,22380.8975,451.4966
0110,คณะมนุษยศาสตร์ สาขาวิชาภาษาตะวันออก กลุ่มวิชาภาษาจีนธุรกิจ ภาคพิเศษ,มหาวิทยาลัยเกษตรศาสตร์,261,40,40,22764.4500,20836.9500,21430.7825,562.3284
0111,คณะมนุษยศาสตร์ สาขาวิชาภาษาตะวันออก กลุ่มวิชาภาษาญี่ปุ่นธุรกิจ ภาคพิเศษ,มหาวิทยาลัยเกษตรศาสตร์,259,40,40,24242.5500,21113.7500,21873.2275,744.9539
0112,คณะวนศาสตร์ สาขาวิชาวนศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,761,120,120,17368.1500,14346.2500,15009.3396,554.1930
0113,คณะวนศาสตร์ สาขาวิชาวิทยาศาสตร์และเทคโนโลยีทางไม้,มหาวิทยาลัยเกษ

In [44]:
def save_result(results, filename):
    header = ['รหัส','คณะ/สาขา','มหาวิทยาลัย','สมัคร','รับ','ผ่าน','สูงสุด','ต่ำสุด','เฉลี่ย','SD']
    writer = csv.writer(open(filename,'w'))
    writer.writerow(header)
    for r in results:
        writer.writerow(extract_line(r))

In [45]:
save_result(results,output_filename)

error ['คณะวศวกรรมศาสตร', 'สาขาวชาวศวกรรมศาสตร', 'จฬาลงกรณมหาวทยาลย']
error ['คณะบรหารธรกจ', 'สาขาวชาบญช', 'มหาวทยาลยเกษตรศาสตร']
error ['คณะประมง', 'มหาวทยาลยเกษตรศาสตร']
error ['คณะศลปศาสตรและวทยาศาสตร', 'สาขาวชาภาษาองกฤษ', 'วทยาเขตก', 'าแพงแสน', 'มหาวทยาลยเกษตรศาสตร']
error ['คณะมนษยศาสตรฯ', 'สาขาวชาภาษาองกฤษ', 'เลอกสอบวชาภาษาฝรงเศส', '(รปแบบท', '2)', 'มหาวทยาลยขอนแกน', '10', '0', '0', 'ลขสทธ', 'สมาคมอธการบดแหงประเทศไทย', 'หามจ', 'าหนาย', 'หนา', '17', 'ตารางแสดงสถตจานวนผสมคร', 'จานวนรบ', 'จานวนผผานฯ', 'และคะแนนสงสด-ตาสด', 'ของการคดเลอกบคคลเขาศกษาในสถาบนอดมศกษาระบบกลาง', '(Admissions)', 'ประจาปการศกษา']
error ['คณะมนษยศาสตรฯ', 'สาขาวชาภาษาองกฤษธรกจ', 'เลอกสอบวชาภาษาฝรงเศส', '(รปแบบท', '2)', 'มหาวทยาลยขอนแกน', '5', '0', '0', 'ลขสทธ', 'สมาคมอธการบดแหงประเทศไทย', 'หามจ', 'าหนาย', 'หนา', '18', 'ตารางแสดงสถตจานวนผสมคร', 'จานวนรบ', 'จานวนผผานฯ', 'และคะแนนสงสด-ตาสด', 'ของการคดเลอกบคคลเขาศกษาในสถาบนอดมศกษาระบบกลาง', '(Admissions)', 'ประจาปการศกษา']
error ['วทยาลยนานาชาต', 'สาขาวชาการตลาดระหว